### Imports necessary for running the program

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 23:01:42 2019

@author: Ahmed
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd
'''df = pd.read_excel("MLDataSet.xlsx")
df2 = pd.read_excel("ADRs.xlsx")
df3 = pd.read_excel("DS.xlsx")
df4 = pd.read_excel("Mental.xlsx")

df5=[df, df2,  df3, df4]

result = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'''
#result.to_excel('result.xlsx')


'df = pd.read_excel("MLDataSet.xlsx")\ndf2 = pd.read_excel("ADRs.xlsx")\ndf3 = pd.read_excel("DS.xlsx")\ndf4 = pd.read_excel("Mental.xlsx")\n\ndf5=[df, df2,  df3, df4]\n\nresult = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'

In [39]:
def removeColumns(result):
    del result['Pain']
    del result['Content']
    del result['Filtered']
    del result['Stemmed']
    del result['big1']
    del result['big2']
    del result['small1']
    del result['small2']
    del result['Height']
    del result['Joined']
    del result['Posted']
    

In [40]:
def vectorizeCount(result):
    for i in range(len(result)):
        if result.at[i,'ADRCount']==0:
            result.at[i,'ADRCount']=0
        if result.at[i,'ADRCount']>0 and result.at[i,'ADRCount']<=3:
            result.at[i,'ADRCount']=1
        if result.at[i,'ADRCount']>3:
            result.at[i,'ADRCount']=2

        if result.at[i,'MentalCount']==0:
            result.at[i,'MentalCount']=0
        if result.at[i,'MentalCount']>0 and result.at[i,'MentalCount']<=3:
            result.at[i,'MentalCount']=1
        if result.at[i,'MentalCount']>3:
            result.at[i,'MentalCount']=2


        if result.at[i,'DieaseCount']==0:
            result.at[i,'DieaseCount']=0
        if result.at[i,'DieaseCount']>0 and result.at[i,'DieaseCount']<=3:
            result.at[i,'DieaseCount']=1
        if result.at[i,'DieaseCount']>3:
            result.at[i,'DieaseCount']=2

### manipulating data to prepare for classification

In [41]:
def manipulate(result,labelfile,label):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    result['Gender']=imp_mean.fit_transform(result['Gender'].values.reshape(-1, 1))
    for i in range(len(result)):
        if result.at[i,'Gender']=='Male':
            result.at[i,'Gender']=0
        if result.at[i,'Gender']=='Female':
            result.at[i,'Gender']=1
    result['Drug']=LabelEncoder().fit_transform(result['Drug'])
    if 'Unnamed: 0' in result:
        del result['Unnamed: 0'] 
    result['DrugFamily']=LabelEncoder().fit_transform(result['DrugFamily'])
    labels=labelfile.iloc[:][label]
    
    #removeColumns(result)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
    result['Age']=imp_mean.fit_transform(result['Age'].values.reshape(-1, 1))
    if 'Height' in result:
        result['Height']=imp_mean.fit_transform(result['Height'].values.reshape(-1, 1))
    return result,labels

### Datasets

In [47]:
label='Hypertensive disease '
result=pd.read_excel('result_reduced.xlsx')
weighted=pd.read_excel('Weighted_reduced.xlsx')
ask=pd.read_excel('askapatient_reduced.xlsx')

resultlabel=pd.read_excel('resultlabels.xlsx')
weightedlabel=pd.read_excel('Weightedlabels.xlsx')
asklabel=pd.read_excel('asklabels.xlsx')

result,result_labels=manipulate(result,resultlabel,label)
weighted,weighted_labels=manipulate(weighted,weightedlabel,label)
ask,ask_labels=manipulate(ask,asklabel,label)
del result['weights2']
del result['Height']


In [5]:
'''for i in weighted:
    if i not in result:
        if i != 'weights2':
            del weighted[i]
weighted.to_excel('Weighted.xlsx')'''

"for i in weighted:\n    if i not in result:\n        if i != 'weights2':\n            del weighted[i]\nweighted.to_excel('Weighted.xlsx')"

In [43]:
ask_labels

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      1
8      0
9      1
10     1
11     1
12     1
13     0
14     0
15     0
16     1
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     1
27     1
28     1
29     0
      ..
726    0
727    0
728    0
729    0
730    1
731    0
732    1
733    1
734    0
735    0
736    0
737    0
738    0
739    0
740    0
741    0
742    0
743    0
744    0
745    1
746    0
747    0
748    0
749    0
750    0
751    0
752    0
753    0
754    0
755    0
Name: Pain, Length: 756, dtype: int64

In [6]:
'''for i in weighted:
    print (weighted[i])'''

'for i in weighted:\n    print (weighted[i])'

In [7]:
def get_score(model, X_train,X_test,Y_train,Y_test):
    model.fit(X_train,Y_train)
    return model.score(X_test,Y_test)

### 10-Fold Cross validation

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
cross_val_score(RandomForestClassifier(n_estimators=400),result,result_labels,cv=10)

array([0.65384615, 0.75641026, 0.78205128, 0.78846154, 0.8525641 ,
       0.76923077, 0.76282051, 0.87179487, 0.73548387, 0.78571429])

In [9]:
#This was used for some other no longer existing purposes
def getMissingPercentage(feature):
    #nancount = int(result[result[feature].isnull()][feature].shape[0])
    nancount = int(result[result[feature]==1][feature].shape[0])
    size=int(result.shape[0])
    print (nancount)
    print ((nancount*100)/size)
#getMissingPercentage('Pvc')

In [18]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10)

scores_rf=[]

for train_index, test_index in kf.split(result):
    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))

In [57]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10)

scores_rf=[]

for train_index, test_index in kf.split(result):
    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]
    clf = RandomForestClassifier(n_estimators=2000, max_depth=2,random_state=0)
    clf.fit(X_train, Y_train)
    pred=clf.predict(X_test)
    
    scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))

C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [31]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0], dtype=int64)

In [59]:
scores_rf


[(0.1762820512820513, 0.5, 0.26066350710900477, None),
 (0.4474206349206349, 0.44667113868857955, 0.44173419439759776, None),
 (0.5694444444444444, 0.549513120977059, 0.5041581399399138, None),
 (0.4595588235294118, 0.4815529096092571, 0.3725312966059565, None),
 (0.4772727272727273, 0.4988400994200497, 0.3226053639846743, None),
 (0.5584299732381802, 0.5467857142857143, 0.5432363849307047, None),
 (0.5243949289281598, 0.5110955792416565, 0.4700854700854701, None),
 (0.5648769574944071, 0.5097250167672703, 0.3580808080808081, None),
 (0.48655913978494625, 0.4870833333333333, 0.483355132694823, None),
 (0.5113762486126526, 0.51956106870229, 0.4902736748230311, None)]